In [445]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
%matplotlib inline
plt.style.use('ggplot')

# Pre-Processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, PolynomialFeatures

# Metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, ShuffleSplit
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


pd.set_option('display.max_columns', None)


## Documentation

For a comprehensive list describing all the features/statistics in this notebook; refer to this documentation link from <a href = 'https://library.fangraphs.com/offense/offensive-statistics-list/'> FanGraphs </a> for basic and advanced statistics descriptions. 

## Batting Data

The `batting_stats` function returns basic and advanced season-level batting data from FanGraphs. 

While this query should work for any historical season, some of the more modern stats (contact %, zone %, and many others) will not be available before certain dates. 

*A note about the statistics being queried: most of the advanced level statistics will not be available before 2014 because <a href = 'https://www.mlb.com/glossary/statcast'> StatCast </a> was not introduced yet.*

In [446]:
from pybaseball import batting_stats

In [447]:
# retrieve data on only players who have 50+ plate appearances between 1990 and 2021
# batting_2021 = batting_stats(2000, 2021, qual=50)

In [448]:
# load in csv
batting_2021 = pd.read_csv('Data/batting_2021', index_col = 0)

In [449]:
batting_2021.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,31,2,46,117,110,198,68,47,9,2,0,4,9,2,0.370,109.0,169.0,80.0,16.0,2398.0,1336.0,1062.0,5.0,0.0,0.0,0.324,0.077,4.21,0.582,0.799,1.381,0.429,0.330,0.0064,0.223,0.304,0.472,0.095,0.272,0.046,0.0,0.544,110.0,183,110.1,4.3,18.6,-6.3,126.8,12.7,$49.5,3.9,244,10.51,-5.90,16.40,118.51,12.63,0.98,3.23,4,11.82,-1.11,0.678,89.0,0.108,79.9,NaN,NaN,0.081,74.2,0.107,76.5,0.026,80.5,NaN,NaN,0.081,NaN,67.8,12.5,NaN,5.1,19.2,0.8,NaN,4.54,5.27,NaN,2.84,8.13,1.37,NaN,0.113,0.702,0.366,0.401,0.899,0.811,0.430,0.453,0.063,-1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,0.9,-4.4,37 - 37,108.9,1.2,2.3,0.447,0.310,0.243,0.095,0.522,0.383,0.475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139,347,47,171,189,273,111,1.05,70.0,135.0,249.0,105.0,106.0,87.0,64.0,83.0,171.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN
2,1109,2001,Barry Bonds,SFG,36,153,476,664,156,49,32,2,73,129,137,177,35,93,9,2,0,5,13,3,0.328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.267,0.140,1.90,0.515,0.863,1.379,0.536,

In [450]:
batting_2021.describe()

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
count,1.200600e+04,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.00000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,12006.000000,12006.000000,12006.000000,12006.000000,11097.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.00000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,12006.000000,10104.000000,12006.000000,12006.000000,12006.000000,10901.000000,10901.000000,10901.000000,10901.000000,9713.000000,9689.000000,10901.000000,10901.000000,10898.000000,10895.000000,10094.000000,10028.000000,4193.000000,4077.000000,8585.000000,1622.000000,10901.000000,10901.000000,9713.000000,10901.000000,10898.000000,10094.000000,4193.000000,10901.000000,10901.000000,9713.000000,10901.000000,10898.000000,10094.000000,4193.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,10901.000000,12006.000000,8193.000000,6683.000000,8127.000000,7477.000000,1404.000000,8028.000000,8191.000000,8189.000000,7263.000000,1493.000000,8186.000000,469.000000,2802.000000,0.0,8193.000000,6683.000000,8127.000000,7477

## bWAR Batting

Get Baseball Reference's WAR stats from its war_daily_bat table, along with some other data that's not included in the batting_stats_bref function. 

For the purposes of this project, we will care about salary, WAR, and the above average columns. We can later merge this dataframe with the `batting_2022` dataframe.

In [451]:
from pybaseball import bwar_bat

# get war stats from baseball reference 
bwar_bat = bwar_bat()
bwar_bat.head()

,name_common,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,pitcher,G,PA,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR
0,David Aardsma,430911.0,aardsda01,2004,SFG,1,NL,Y,11,0.0,300000.0,0.0,0.0,0.0,0.0,0.00,0.00
1,David Aardsma,430911.0,aardsda01,2006,CHC,1,NL,Y,43,3.0,NaN,-0.4,-0.4,0.0,0.0,-0.04,-0.04
2,David Aardsma,430911.0,aardsda01,2007,CHW,1,AL,Y,2,0.0,387500.0,0.0,0.0,0.0,0.0,0.00,0.00
3,David Aardsma,430911.0,aardsda01,2008,BOS,1,AL,Y,5,1.0,403250.0,-0.2,-0.2,0.0,0.0,-0.02,-0.02
4,David Aardsma,430911.0,aardsda01,2009,SEA,1,AL,Y,3,0.0,419000.0,0.0,0.0,0.0,0.0,0.00,0.00


In [452]:
# load in csv
bwar_bat = pd.read_csv('Data/bwar_bat', index_col = 0)

In [453]:
# filter bwar_bat to 2000 - 2021 and minimum 50 PA
bwar_bat_2021 = bwar_bat[(bwar_bat['year_ID'].between(2000,2021)) & (bwar_bat['PA'] >= 50)]
bwar_bat_2021.sort_values(by = 'year_ID')

,name_common,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,pitcher,G,PA,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR
109,Jeff Abbott,110010.0,abbotje01,2000,CHW,1,AL,N,80,242.0,255000.0,-12.6,-6.7,-7.3,0.84,-1.13,-0.29
86775,Tom Prince,120771.0,princto01,2000,PHI,1,NL,N,46,140.0,300000.0,-4.2,-4.2,2.3,0.42,-0.44,-0.02
33254,Robert Fick,137001.0,fickro01,2000,DET,1,AL,N,66,188.0,200000.0,-5.3,-5.6,-1.8,0.65,-0.47,0.18
32714,Carlos Febles,136866.0,febleca01,2000,KCR,1,AL,N,100,399.0,250000.0,2.8,-3.6,8.9,1.39,0.31,1.70
32656,Sal Fasano,114031.0,fasansa01,2000,OAK,1,AL,N,52,144.0,240000.0,1.0,0.0,3.5,0.50,0.11,0.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96813,Kyle Schwarber,656941.0,schwaky01,2021,BOS,2,AL,N,41,168.0,NaN,7.5,10.5,-5.4,0.56,0.77,1.33
96814,Kyle Schwarber,656941.0,schwaky01,2021,WSN,1,NL,N,72,303.0,7000000.0,8.8,13.8,-7.5,1.02,0.81,1.83
96832,Frank Schwindel,643524.0,schwifr01,2021,CHC,2,NL,N,56,239.0,NaN,10.1,14.1,-7.1,0.80,0.95,1.75
23298,Jake Cronenworth,630105.0,croneja01,2021,SDP,1,NL,N,152,643.0,NaN,27.7,22.7,8.5,2.16,2.64,4.80


In [454]:
bwar_bat_2021.describe()

,mlb_ID,year_ID,stint_ID,G,PA,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR
count,12616.000000,12616.000000,12616.000000,12616.000000,12616.000000,9.354000e+03,12616.000000,12616.000000,12616.000000,12616.000000,12616.000000,12616.000000
mean,378575.647987,2010.391011,1.071100,84.781706,305.154011,4.163878e+06,0.570799,0.563475,0.002949,0.982166,0.044187,1.026352
std,180548.349417,6.296166,0.269057,45.605022,206.944329,5.419254e+06,13.837667,12.055243,7.627333,0.700321,1.399351,1.776108
min,110010.000000,2000.000000,1.000000,12.000000,50.000000,1.000000e+00,-50.400000,-42.400000,-50.700000,0.000000,-5.040000,-3.250000
25%,150148.000000,2005.000000,1.000000,41.000000,115.000000,5.104250e+05,-7.300000,-6.300000,-3.700000,0.380000,-0.750000,-0.140000
50%,430965.000000,2010.000000,1.000000,81.000000,253.000000,1.750000e+06,-1.600000,-1.600000,0.100000,0.820000,-0.170000,0.460000
75%,506942.250000,2016.000000,1.000000,128.000000,482.000000,5.900000e+06,5.100000,4.400000,3.300000,1.580000,0.520000,1.720000
max,683734.000000,2021.000000,4.000000,163.000000,778.000000,3.833333e+07,106.300000,110.900000,49.300000,2.520000,9.880000,11.850000


In [455]:
# how many missing values in batting salary column?
print("Missing bWAR Batting Salary Data:")
bwar_bat_2021['salary'].isna().value_counts(normalize = True)

Missing bWAR Batting Salary Data:


False    0.741439
True     0.258561
Name: salary, dtype: float64

## Pitching Data

The `pitching_stats` function returns basic and advanced season-level pitching data from FanGraphs.

While this query should work for any historical season, some of the more modern stats (contact %, zone %, and many others) will not be available before certain dates.

In [456]:
from pybaseball import pitching_stats

In [457]:
# retrieve data on only players who have pitched 20+ innings since 2000
pitching_2021 = pitching_stats(2000, 2021, qual=20)

In [458]:
# load in csv
pitching_2021 = pd.read_csv('Data/pitching_2021', index_col = 0)

In [459]:
pitching_2021.head()

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,Dollars,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,Clutch,FB% 2,FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,HLD,SD,MD,ERA-,FIP-,xFIP-,K%,BB%,SIERA,RS/9,E-F,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,RA9-WAR,BIP-Wins,LOB-Wins,FDP-Wins,Age Rng,K-BB%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,kwERA,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,K/9+,BB/9+,K/BB+,H/9+,HR/9+,AVG+,WHIP+,BABIP+,LOB%+,K%+,BB%+,LD%+,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
910,60,2001,Randy Johnson,ARI,37,21,6,10.4,2.49,35,34,3,2,0,NaN,249.2,994,181,74,69,19,71,2,18,8,1,372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.41,2.56,5.24,6.52,0.68,0.200,1.01,0.315,0.805,2.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.4,242.2,4.5,7.0,90.9,NaN,NaN,NaN,6.23,-14.23,20.46,64.42,6.25,1.04,0.98,0.88,1.07,31,5.84,0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,55,47,NaN,0.374,0.071,NaN,NaN,0.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.8,-1.0,0.5,-0.6,37 - 37,0.303,NaN,NaN,NaN,NaN,NaN,NaN,1.85,0.465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192,77,249,73,59,78,74,108,112,208,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
1152,60,2000,Randy Johnson,ARI,36,19,7,9.6,2.64,35,35,8,3,0,NaN,248.2,1001,202,89,73,23,76,1,6,5,2,347,NaN,NaN,NaN,Na

In [460]:
pitching_2021.describe()

,IDfg,Season,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,Clutch,FB% 2,FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,HLD,SD,MD,ERA-,FIP-,xFIP-,K%,BB%,SIERA,RS/9,E-F,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,RA9-WAR,BIP-Wins,LOB-Wins,FDP-Wins,K-BB%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,kwERA,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,K/9+,BB/9+,K/BB+,H/9+,HR/9+,AVG+,WHIP+,BABIP+,LOB%+,K%+,BB%+,LD%+,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
count,1.051900e+04,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,9638.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,5824.000000,5824.000000,7711.000000,7711.000000,10519.000000,9638.000000,9638.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10519.000000,10518.000000,10519.000000,10519.000000,10519.000000,9628.000000,9628.000000,7919.000000,7912.000000,2618.000000,2584.000000,6805.000000,6765.000000,8460.000000,8455.000000,1505.000000,1463.000000,52.000000,50.00000,8263.000000,1527.000000,9628.000000,7919.000000,2618.000000,6805.000000,8460.000000,1505.000000,52.000000,9628.000000,7919.000000,2618.000000,6805.000000,8460.000000,1505.000000,52.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,9638.000000,10519.000000,10519.000000,10519.000000,10519.000000,9638.000000,10519.000000,10519.000000,9638.000000,9638.0

## bWAR Pitching 

Get Baseball Reference's WAR stats from its war_daily_pitch table, along with some other data that's not included in the pitching_stats_bref function. 

For the purposes of this project, we will care about salary, WAR, and the above average columns. We can later merge this dataframe with the `pitching_2022` dataframe.

In [461]:
from pybaseball import bwar_pitch

# get war stats from baseball reference 
bwar_pitch = bwar_pitch(return_all = True)
bwar_pitch.head()

,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,G,GS,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg
0,David Aardsma,22.0,430911.0,aardsda01,2004,SFG,1,NL,11,0,32,0.0,32.0,8,5.967,-0.440,NaN,-0.423,101,98.525,5.862,44.0,0.0093,-45.5,-2.138,-2.177,-1.199,0.203,1.0718,-0.15,300000.0,4.67092,4.86883,1.902,0.4803,-0.2167,-0.0298,4.75633,1.895,0.4914,0.0942,66.675000,5.334
1,David Aardsma,24.0,430911.0,aardsda01,2006,CHC,1,NL,45,0,159,0.0,159.0,25,28.172,-2.076,NaN,-0.367,103,104.018,27.526,147.0,0.0337,-10.9,2.526,2.751,7.388,0.211,0.7542,0.61,NaN,4.86675,4.80561,1.909,0.5060,0.2700,-0.1301,4.97303,1.919,0.4896,0.4682,114.295833,27.431
2,David Aardsma,25.0,430911.0,aardsda01,2007,CHW,1,AL,25,0,97,0.0,97.0,24,17.999,-1.343,NaN,-1.327,104,103.205,18.559,97.0,0.0206,-64.4,-5.441,-5.646,-1.814,0.218,1.4280,-0.40,387500.0,4.85895,5.08479,1.924,0.4782,-0.5450,-0.1990,5.00394,1.920,0.4859,0.3459,74.086957,17.040
3,David Aardsma,26.0,430911.0,aardsda01,2008,BOS,1,AL,47,0,146,0.0,146.0,32,26.131,-1.942,NaN,0.663,106,106.614,25.082,139.0,0.0317,20.9,-6.918,-7.757,-1.633,0.211,0.7972,-0.31,403250.0,4.70400,4.86905,1.904,0.4836,-0.7708,-0.0632,4.81624,1.901,0.4888,0.5205,83.796667,25.139
4,David Aardsma,27.0,430911.0,aardsda01,2009,SEA,1,AL,73,0,214,0.0,214.0,23,39.221,-2.895,NaN,3.136,95,93.595,31.064,181.0,0.0399,78.6,8.064,7.689,15.965,0.215,1.8922,1.69,419000.0,4.79788,4.69256,1.899,0.5105,0.7665,0.1577,4.90529,1.911,0.4894,0.7666,168.910000,33.782


In [462]:
# load in csv
bwar_pitch = pd.read_csv('Data/bwar_pitch', index_col = 0)

In [463]:
# filter bwar_bat to 2000 - 2021
bwar_pitch_2021 = bwar_pitch[(bwar_pitch['year_ID'].between(2000, 2021))]
bwar_pitch_2021.sort_values(by = 'year_ID')

,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,G,GS,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg
48453,Ismael Valdez,26.0,123595.0,valdeis01,2000,LAD,2,NL,9,8,120,113.0,7.0,29,21.189,0.622,NaN,0.193,93,90.888,19.648,135.0,0.0306,6.3,-9.352,-9.470,-5.544,0.219,0.0100,-0.49,NaN,5.05048,6.10270,1.988,0.4070,-0.8370,-0.0028,5.46714,1.955,0.4613,0.3455,71.618519,19.337
36746,Brad Penny,22.0,207267.0,pennybr01,2000,FLA,1,NL,23,22,359,348.0,11.0,70,65.451,2.124,NaN,-0.977,95,95.346,65.362,384.0,0.0842,-11.6,-4.638,-4.992,6.753,0.219,0.6600,0.48,200000.0,5.05048,5.26752,1.945,0.4796,-0.4692,-0.0736,5.53825,1.959,0.4550,1.0266,91.789062,58.745
15902,Aaron Fultz,26.0,150269.0,fultzaa01,2000,SFG,1,NL,58,0,208,0.0,208.0,38,39.600,-3.033,NaN,0.259,92,95.699,34.746,206.0,0.0455,5.7,-3.254,-3.459,3.346,0.219,1.0952,0.12,200000.0,5.05048,5.11011,1.936,0.4943,-0.3306,-0.1557,5.16254,1.939,0.4894,0.6099,91.766667,33.036
36671,Jesus Pena,25.0,227494.0,penaje01,2000,CHW,1,AL,20,0,70,0.0,70.0,18,13.931,-1.031,NaN,0.088,102,103.505,13.261,73.0,0.0158,5.6,-4.739,-4.915,-1.874,0.239,0.5335,-0.15,NaN,5.31786,5.56361,1.974,0.4777,-0.4460,0.0458,5.46245,1.969,0.4868,0.2505,94.178571,13.185
36670,Jesus Pena,25.0,227494.0,penaje01,2000,BOS,2,AL,2,0,9,0.0,9.0,1,1.685,-0.125,NaN,0.082,103,102.143,1.510,10.0,0.0022,37.3,0.510,0.487,0.878,0.239,1.2250,0.08,NaN,5.31786,5.07436,1.949,0.5228,0.0456,-0.0024,5.50376,1.971,0.4831,0.0321,186.000000,1.860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16240,Luis Garcia,24.0,677651.0,garcilu05,2021,HOU,1,AL,30,28,466,448.0,18.0,62,80.240,3.471,0.000,7.563,99,98.900,75.310,413.0,0.1036,73.0,13.310,12.334,28.486,0.207,0.3850,2.62,NaN,4.68464,4.27350,1.868,0.5428,1.2840,-0.1395,5.19490,1.921,0.4505,1.4709,123.480000,74.088
2190,Joe Barlow,25.0,669618.0,barlojo01,2021,TEX,1,AL,31,0,87,0.0,87.0,9,15.034,-0.976,1.252,1.579,101,100.216,13.761,72.0,0.0168,94.0,4.761,4.579,7.594,0.207,1.2455,0.74,NaN,4.68464,4.53693,1.884,0.5151,0.4681,-0.0104,4.77683,1.897,0.4908,0.2826,284.980000,14.249
34785,Darien Núñez,28.0,628345.0,nunezda01,2021,LAD,1,NL,6,1,23,0.0,23.0,8,3.851,-0.251,0.000,0.226,97,96.424,3.253,21.0,0.0055,41.0,-4.747,-4.728,-4.016,0.202,1.0860,-0.42,NaN,4.57192,5.35992,1.924,0.4241,-0.4554,-0.0408,4.69474,1.886,0.4875,0.0752,53.100000,3.717
31583,Julian Merryweather,29.0,657240.0,merryju01,2021,TOR,1,AL,13,1,39,3.0,36.0,7,6.608,-0.372,1.256,0.233,101,99.673,7.235,38.0,0.0097,24.0,0.235,0.153,1.505,0.207,1.2650,0.12,NaN,4.68464,4.67287,1.891,0.5012,0.0156,-0.0269,4.78319,1.898,0.4901,0.1275,94.371429,6.606


In [464]:
bwar_pitch_2021.describe()

,age,mlb_ID,year_ID,stint_ID,G,GS,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg
count,16555.000000,16555.000000,16555.000000,16555.000000,16555.000000,16555.000000,16555.000000,16520.000000,16520.000000,16555.000000,16555.000000,16555.000000,1808.000000,16555.000000,16555.000000,16520.000000,16555.000000,16555.000000,16555.000000,16555.000000,16555.000000,16555.000000,16555.000000,16555.000000,16555.000000,16520.000000,8.854000e+03,16555.000000,16520.000000,16520.000000,16520.000000,16520.000000,16555.000000,16520.000000,16520.000000,16520.000000,16520.000000,15959.000000,16555.000000
mean,28.208819,420390.276170,2011.200725,1.091755,24.818665,6.271821,167.713259,108.986864,59.081719,28.677439,28.863240,-0.008486,0.297436,0.129360,100.080882,100.381012,28.792937,171.880519,0.039867,2.601963,0.115498,0.000002,5.479490,0.203146,0.902624,0.530495,3.114571e+06,4.601472,4.805057,1.892145,0.484686,0.037665,-0.037586,4.847261,1.895817,0.475700,0.530482,107.670597,26.621117
std,4.092570,169844.593075,6.424128,0.303380,20.692657,10.366798,173.009266,188.530744,69.330749,28.686131,29.714675,1.805132,0.650680,2.155056,5.036742,5.904402,30.839335,179.299934,0.040367,38.258688,9.936283,9.887849,13.078215,0.013243,0.476755,1.309606,4.556342e+06,0.277473,0.871441,0.055473,0.070806,1.026784,0.113129,0.359051,0.035007,0.019151,0.540357,69.273763,27.399424
min,19.000000,110015.000000,2000.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-15.000000,0.000000,0.000000,-4.400000,0.000000,-15.587000,88.000000,76.000000,0.000000,0.000000,0.000000,-115.200000,-43.678000,-44.187000,-35.371000,0.175000,0.000000,-2.760000,1.000000e+05,3.990350,1.000000,1.639000,0.069500,-3.744900,-0.756000,4.017010,1.809000,0.418000,0.000000,2.428571,0.170000
25%,25.000000,339492.000000,2006.000000,1.000000,8.000000,0.000000,38.000000,0.000000,3.000000,8.000000,6.618500,-1.107000,0.000000,-0.432000,97.000000,96.800000,6.308000,40.000000,0.009700,-24.000000,-4.821000,-4.893500,-2.082000,0.195000,0.600000,-0.200000,4.935000e+05,4.410520,4.335300,1.858000,0.460200,-0.468375,-0.104600,4.599827,1.874000,0.457700,0.122600,71.773028,6.129000
50%,28.000000,452291.000000,2012.000000,1.000000,20.000000,0.000000,110.000000,0.000000,28.000000,19.000000,18.981000,-0.226000,0.000000,0.028000,100.000000,100.000000,18.286000,111.000000,0.026700,3.000000,-0.661000,-0.710000,1.846000,0.203000,1.000000,0.120000,1.050000e+06,4.641860,4.679350,1.889000,0.495400,-0.072950,-0.045000,4.829575,1.899000,0.486800,0.351800,95.935714,17.398000
75%,31.000000,543488.000000,2017.000000,1.000000,34.000000,9.000000,218.000000,136.000000,96.000000,37.000000,37.784000,0.678000,0.000000,0.617500,103.000000,103.068000,36.628000,220.000000,0.050700,27.300000,3.572000,3.489000,9.413500,0.211000,1.134500,0.870000,4.000000e+06,4.797880,5.086750,1.919000,0.517000,0.364000,-0.001000,5.072147,1.920000,0.490600,0.702200,125.360399,34.800000
max,49.000000,685503.000000,2021.000000,5.000000,94.000000,36.000000,798.000000,798.000000,316.000000,152.000000,145.076000,5.806000,5.611000,16.916000,125.000000,150.000000,162.764000,826.000000,0.180800,116.000000,93.600000,91.966000,120.240000,0.239000,7.370000,11.710000,4.000000e+07,5.317860,14.307230,2.313000,0.926100,9.662800,1.317300,6.245200,2.009000,0.500000,2.691100,1374.900000,139.400000


In [465]:
# how many missing values in pitching salary column?
print("Missing bWAR Pitching Salary Data:")
bwar_pitch_2021['salary'].isna().value_counts(normalize = True)

Missing bWAR Pitching Salary Data:


False    0.534823
True     0.465177
Name: salary, dtype: float64

## Formatting the Batting Table

**Join `batting_2021` with `bwar_bat_2021` on the following columns:**

**`batting_2021`** 
- `Name`
- `Season`

**`bwar_bat_2021`**  
- `name_common`
- `year_ID`

In [466]:
# join batting_2021 onto bwar_bat_2021 (left join)
batting_merged = pd.merge(bwar_bat_2021, batting_2021, how='left',left_on=['name_common','year_ID'],right_on=['Name','Season'])

In [467]:
batting_merged.head(10)

,name_common,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,pitcher,G_x,PA_x,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR_x,IDfg,Season,Name,Team,Age,G_y,AB,PA_y,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR_y,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,Jeff Abbott,110010.0,abbotje01,2000,CHW,1,AL,N,80,242.0,255000.0,-12.6,-6.7,-7.3,0.84,-1.13,-0.29,1000011.0,2000.0,Jeff Abbott,CHW,27.0,80.0,215.0,242.0,59.0,40.0,15.0,1.0,3.0,31.0,29.0,21.0,1.0,38.0,2.0,1.0,2.0,2.0,2.0,1.0,0.274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087,0.157,0.55,0.343,0.395,0.738,0.121,0.320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.328,-2.7,29.0,-4.2,-5.0,7.8,-1.6,-2.1,-0.2,NaN,3.9,87.0,0.37,-3.89,4.26,-6.74,-0.57,0.90,1.79,14.0,-0.16,0.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.6,0.0,NaN,27 - 27,-4.2,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0,91.0,100.0,98.0,89.0,72.0,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

**Filling missing `salary` values.**

We have missing salary values for approximately 25% of batters in the `batting_merged` dataset. Most of the missing values are likely due to clerical input whereby a player's salary for future years may not be filled in, but may be accounted for in an earlier year. 

For example, take a look at the player Reggie Abercrombie in the 2006 season. His 2006 salary is available, however, 2007 season is not. And in 2008, Reggie changed teams and his salary is still missing. Similarly, we see missing values for Brent Abernathy, whereby his 2001 and 2005 seasons contain missing salary data but his salary is available in the 2002 season. 

There are also players who were traded mid-season. The player is accounted for twice for a single season because they played for a different team and the salary may be missing for one of the entries. However, we will treat their salaries as the same for both teams. 

To fill in these missing values, we will assume the following:
- A player's salary is the same across consecutive seasons even if the salary is missing
- Players traded mid-season will have the same salary for that season across both teams


In [468]:
# make a copy and specify which column to fill missing values
salary_col = ['salary']
batting_filled = batting_merged.copy()

In [469]:
# forward and backward fills missing values grouped by name
batting_filled.loc[:,salary_col] = batting_merged.loc[:,salary_col].groupby(batting_filled['name_common']).ffill().bfill()

In [470]:
# check
batting_filled.tail(10)

,name_common,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,pitcher,G_x,PA_x,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR_x,IDfg,Season,Name,Team,Age,G_y,AB,PA_y,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR_y,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
12644,Julio Zuleta,275936.0,zuletju01,2001,CHC,1,NL,N,49,118.0,200000.0,-6.7,-5.4,-2.6,0.35,-0.71,-0.36,1014462.0,2001.0,Julio Zuleta,CHC,26.0,49.0,106.0,118.0,23.0,14.0,3.0,0.0,6.0,11.0,24.0,8.0,1.0,32.0,3.0,1.0,0.0,3.0,0.0,1.0,0.217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068,0.271,0.25,0.288,0.415,0.703,0.198,0.246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.301,-2.7,12.0,-3.1,-1.0,3.7,-1.9,-2.6,-0.3,NaN,1.4,80.0,-0.46,-2.49,2.03,0.49,0.09,1.02,0.89,24.0,-0.26,-0.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.9,-0.4,NaN,26 - 26,-3.6,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.0,76.0,158.0,85.0,94.0,116.0,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [471]:
# check
batting_filled.salary.isna().sum()

0

In [472]:
batting_filled.describe()

,mlb_ID,year_ID,stint_ID,G_x,PA_x,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR_x,IDfg,Season,Age,G_y,AB,PA_y,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR_y,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
count,12654.000000,12654.000000,12654.000000,12654.000000,12654.000000,1.265400e+04,12654.000000,12654.000000,12654.000000,12654.000000,12654.000000,12654.000000,1.234000e+04,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,11177.000000,11177.000000,11177.000000,11177.000000,11177.000000,11177.000000,11177.000000,11177.000000,11177.000000,11177.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,11177.000000,11177.000000,11177.000000,11177.000000,11177.000000,11177.000000,11177.000000,11177.000000,12340.000000,12340.000000,12340.000000,12340.000000,11465.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,12340.000000,10488.000000,12340.000000,12340.000000,12340.000000,11177.000000,11177.000000,11177.000000,11177.000000,9961.000000,9939.000000,11177.000000,11177.000000,11174.000000,11171.000000,10417.000000,10353.000000,4400.000000,4275.000000,8903.000000,1692.000000,11177.000000,11177.000000,9961.000000,11177.000000,11174.000000,10417.000000,4400.000000,11177.000000,11177.000000,9961.000000,11177.000000,11174.000000,10417.000000,4400.000000,11177.000000,11177.000000,11177.000000,11177.000000,11

## Simplify and separate the `batting_filled` table

Because there are so many advanced statistics available in the dataframe, we can reduce the number of columns down to standard team statistics that do not include StatCast data. 

As mentioned earlier, StatCast was first introduced to MLB in 2014 and was not fully installed at every ballpark until 2015. Therfore, there will be many missing values for the advanced StatCast features. 

Thus, we will create two separate batting tables:
- `batting_basic` (containing basic data between 2000 - 2021)
- `batting_adv` (containing basic AND StatCast data between 2014 - 2021)

We'll also remove repetitive columns that we brought in during the initial merge of tables. 

In [473]:
batting_filled.head(1)

,name_common,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,pitcher,G_x,PA_x,salary,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR_x,IDfg,Season,Name,Team,Age,G_y,AB,PA_y,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR_y,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,Jeff Abbott,110010.0,abbotje01,2000,CHW,1,AL,N,80,242.0,255000.0,-12.6,-6.7,-7.3,0.84,-1.13,-0.29,1000011.0,2000.0,Jeff Abbott,CHW,27.0,80.0,215.0,242.0,59.0,40.0,15.0,1.0,3.0,31.0,29.0,21.0,1.0,38.0,2.0,1.0,2.0,2.0,2.0,1.0,0.274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087,0.157,0.55,0.343,0.395,0.738,0.121,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.328,-2.7,29.0,-4.2,-5.0,7.8,-1.6,-2.1,-0.2,NaN,3.9,87.0,0.37,-3.89,4.26,-6.74,-0.57,0.9,1.79,14.0,-0.16,0.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.6,0.0,NaN,27 - 27,-4.2,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0,91.0,100.0,98.0,89.0,72.0,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.

In [474]:
# get all column names
with pd.option_context('display.max_seq_items', None):
    print (batting_filled.columns)

Index(['name_common', 'mlb_ID', 'player_ID', 'year_ID', 'team_ID', 'stint_ID',
       'lg_ID', 'pitcher', 'G_x', 'PA_x', 'salary', 'runs_above_avg',
       'runs_above_avg_off', 'runs_above_avg_def', 'WAR_rep', 'WAA', 'WAR_x',
       'IDfg', 'Season', 'Name', 'Team', 'Age', 'G_y', 'AB', 'PA_y', 'H', '1B',
       '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH',
       'GDP', 'SB', 'CS', 'AVG', 'GB', 'FB', 'LD', 'IFFB', 'Pitches', 'Balls',
       'Strikes', 'IFH', 'BU', 'BUH', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS',
       'ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%',
       'BUH%', 'wOBA', 'wRAA', 'wRC', 'Bat', 'Fld', 'Rep', 'Pos', 'RAR',
       'WAR_y', 'Dol', 'Spd', 'wRC+', 'WPA', '-WPA', '+WPA', 'RE24', 'REW',
       'pLI', 'phLI', 'PH', 'WPA/LI', 'Clutch', 'FB% (Pitch)', 'FBv', 'SL%',
       'SLv', 'CT%', 'CTv', 'CB%', 'CBv', 'CH%', 'CHv', 'SF%', 'SFv', 'KN%',
       'KNv', 'XX%', 'PO%', 'wFB', 'wSL', 'wCT', 'wCB', 'wCH', 'wSF', 'wKN',


**Specify columns for `batting_basic`**

In [475]:
# basic batting columns to keep
batting_basic_columns = ['name_common', 'player_ID', 'year_ID', 'team_ID', 'lg_ID',
                         'salary', 'Age', 'G_y', 'AB', 'PA_y', 'AVG', 'H', '1B', '2B', '3B', 
                         'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 
                         'CS', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS', 'ISO', 'BABIP']

# create new batting_basic table with specified columns i want
batting_basic = batting_filled[batting_basic_columns]

# renaming columns
batting_basic = batting_basic.rename(columns = {'name_common':'name',
                                                'G_y':'G',
                                                'PA_y':'PA'})

**Specify columns for `batting_adv`**

In [476]:
# adv batting columns to keep
batting_adv_columns = ['name_common', 'player_ID', 'year_ID', 'team_ID', 'lg_ID',
                           'salary', 'Age', 'G_y', 'AB', 'PA_y', 'AVG', 'H', '1B', '2B', '3B', 
                           'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 
                           'CS', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS', 'ISO', 'BABIP',
                           'runs_above_avg', 'runs_above_avg_off', 'runs_above_avg_def', 
                           'WAR_rep', 'WAA', 'WAR_x', 'GB', 'FB', 'LD', 'IFFB', 'Pitches', 
                           'Balls', 'Strikes', 'IFH', 'BU', 'BUH', 'BB%', 'K%', 'BB/K', 'OBP', 
                           'SLG', 'OPS', 'ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 
                           'HR/FB', 'IFH%', 'BUH%', 'wOBA', 'wRAA', 'wRC', 'Bat', 'Fld', 'Rep', 
                           'Pos', 'RAR', 'Spd', 'wRC+', 'WPA', '-WPA', '+WPA', 'RE24', 'REW',
                           'pLI', 'phLI', 'PH', 'WPA/LI', 'Clutch', 'FB% (Pitch)', 'FBv', 'SL%',
                           'SLv', 'CT%', 'CTv', 'CB%', 'CBv', 'CH%', 'CHv', 'SF%', 'SFv', 'KN%',
                           'KNv', 'XX%', 'PO%', 'wFB', 'wSL', 'wCT', 'wCB', 'wCH', 'wSF', 'wKN',
                           'wFB/C', 'wSL/C', 'wCT/C', 'wCB/C', 'wCH/C', 'wSF/C', 'wKN/C',
                           'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%', 'Z-Contact%',
                           'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'BsR', 'FA% (sc)',
                           'FT% (sc)', 'FC% (sc)', 'FS% (sc)', 'FO% (sc)', 'SI% (sc)', 'SL% (sc)',
                           'CU% (sc)', 'KC% (sc)', 'EP% (sc)', 'CH% (sc)', 'SC% (sc)', 'KN% (sc)',
                           'UN% (sc)', 'vFA (sc)', 'vFT (sc)', 'vFC (sc)', 'vFS (sc)', 'vFO (sc)',
                           'vSI (sc)', 'vSL (sc)', 'vCU (sc)', 'vKC (sc)', 'vEP (sc)', 'vCH (sc)',
                           'vSC (sc)', 'vKN (sc)', 'FA-X (sc)', 'FT-X (sc)', 'FC-X (sc)',
                           'FS-X (sc)', 'FO-X (sc)', 'SI-X (sc)', 'SL-X (sc)', 'CU-X (sc)',
                           'KC-X (sc)', 'EP-X (sc)', 'CH-X (sc)', 'SC-X (sc)', 'KN-X (sc)',
                           'FA-Z (sc)', 'FT-Z (sc)', 'FC-Z (sc)', 'FS-Z (sc)', 'FO-Z (sc)',
                           'SI-Z (sc)', 'SL-Z (sc)', 'CU-Z (sc)', 'KC-Z (sc)', 'EP-Z (sc)',
                           'CH-Z (sc)', 'SC-Z (sc)', 'KN-Z (sc)', 'wFA (sc)', 'wFT (sc)',
                           'wFC (sc)', 'wFS (sc)', 'wFO (sc)', 'wSI (sc)', 'wSL (sc)', 'wCU (sc)',
                           'wKC (sc)', 'wEP (sc)', 'wCH (sc)', 'wSC (sc)', 'wKN (sc)',
                           'wFA/C (sc)', 'wFT/C (sc)', 'wFC/C (sc)', 'wFS/C (sc)', 'wFO/C (sc)',
                           'wSI/C (sc)', 'wSL/C (sc)', 'wCU/C (sc)', 'wKC/C (sc)', 'wEP/C (sc)',
                           'wCH/C (sc)', 'wSC/C (sc)', 'wKN/C (sc)', 'O-Swing% (sc)',
                           'Z-Swing% (sc)', 'Swing% (sc)', 'O-Contact% (sc)', 'Z-Contact% (sc)',
                           'Contact% (sc)', 'Zone% (sc)', 'Pace', 'Def', 'wSB', 'UBR', 'Age Rng',
                           'Off', 'Lg', 'wGDP', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%',
                           'Hard%', 'TTO%', 'CH% (pi)', 'CS% (pi)', 'CU% (pi)', 'FA% (pi)',
                           'FC% (pi)', 'FS% (pi)', 'KN% (pi)', 'SB% (pi)', 'SI% (pi)', 'SL% (pi)',
                           'XX% (pi)', 'vCH (pi)', 'vCS (pi)', 'vCU (pi)', 'vFA (pi)', 'vFC (pi)',
                           'vFS (pi)', 'vKN (pi)', 'vSB (pi)', 'vSI (pi)', 'vSL (pi)', 'vXX (pi)',
                           'CH-X (pi)', 'CS-X (pi)', 'CU-X (pi)', 'FA-X (pi)', 'FC-X (pi)',
                           'FS-X (pi)', 'KN-X (pi)', 'SB-X (pi)', 'SI-X (pi)', 'SL-X (pi)',
                           'XX-X (pi)', 'CH-Z (pi)', 'CS-Z (pi)', 'CU-Z (pi)', 'FA-Z (pi)',
                           'FC-Z (pi)', 'FS-Z (pi)', 'KN-Z (pi)', 'SB-Z (pi)', 'SI-Z (pi)',
                           'SL-Z (pi)', 'XX-Z (pi)', 'wCH (pi)', 'wCS (pi)', 'wCU (pi)',
                           'wFA (pi)', 'wFC (pi)', 'wFS (pi)', 'wKN (pi)', 'wSB (pi)', 'wSI (pi)',
                           'wSL (pi)', 'wXX (pi)', 'wCH/C (pi)', 'wCS/C (pi)', 'wCU/C (pi)',
                           'wFA/C (pi)', 'wFC/C (pi)', 'wFS/C (pi)', 'wKN/C (pi)', 'wSB/C (pi)',
                           'wSI/C (pi)', 'wSL/C (pi)', 'wXX/C (pi)', 'O-Swing% (pi)',
                           'Z-Swing% (pi)', 'Swing% (pi)', 'O-Contact% (pi)', 'Z-Contact% (pi)',
                           'Contact% (pi)', 'Zone% (pi)', 'Pace (pi)', 'FRM', 'AVG+', 'BB%+',
                           'K%+', 'OBP+', 'SLG+', 'ISO+', 'BABIP+', 'LD+%', 'GB%+', 'FB%+',
                           'HR/FB%+', 'Pull%+', 'Cent%+', 'Oppo%+', 'Soft%+', 'Med%+', 'Hard%+',
                           'EV', 'LA', 'Barrels', 'Barrel%', 'maxEV', 'HardHit', 'HardHit%',
                           'Events', 'CStr%', 'CSW%']

# create new batting_adv table with specified columns i want
batting_adv = batting_filled[batting_adv_columns]

# renaming columns
batting_adv = batting_adv.rename(columns = {'name_common':'name',
                                            'G_y':'G',
                                            'PA_y':'PA',
                                            'WAR_x':'WAR'})

**Checking and cleaning `batting_basic` first:**

In [477]:
# check
batting_basic.head()

,name,player_ID,year_ID,team_ID,lg_ID,salary,Age,G,AB,PA,AVG,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP
0,Jeff Abbott,abbotje01,2000,CHW,AL,255000.0,27.0,80.0,215.0,242.0,0.274,59.0,40.0,15.0,1.0,3.0,31.0,29.0,21.0,1.0,38.0,2.0,1.0,2.0,2.0,2.0,1.0,0.087,0.157,0.55,0.343,0.395,0.738,0.121,0.320
1,Kurt Abbott,abbotku01,2000,NYM,NL,500000.0,31.0,79.0,157.0,173.0,0.217,34.0,20.0,7.0,1.0,6.0,22.0,12.0,14.0,2.0,51.0,1.0,1.0,0.0,2.0,1.0,1.0,0.081,0.295,0.27,0.283,0.389,0.672,0.172,0.277
2,Reggie Abercrombie,abercre01,2006,FLA,NL,327000.0,24.0,111.0,255.0,281.0,0.212,54.0,35.0,12.0,2.0,5.0,39.0,24.0,18.0,2.0,78.0,3.0,1.0,4.0,2.0,6.0,5.0,0.064,0.278,0.23,0.271,0.333,0.604,0.122,0.283
3,Reggie Abercrombie,abercre01,2007,FLA,NL,327000.0,25.0,35.0,76.0,80.0,0.197,15.0,10.0,3.0,0.0,2.0,16.0,5.0,2.0,0.0,22.0,2.0,0.0,0.0,1.0,7.0,1.0,0.025,0.275,0.09,0.238,0.316,0.553,0.118,0.250
4,Reggie Abercrombie,abercre01,2008,HOU,NL,327000.0,26.0,34.0,55.0,60.0,0.309,17.0,10.0,5.0,0.0,2.0,10.0,5.0,1.0,0.0,23.0,2.0,1.0,1.0,0.0,5.0,2.0,0.017,0.383,0.04,0.339,0.509,0.848,0.200,0.484


In [478]:
batting_basic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12654 entries, 0 to 12653
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       12654 non-null  object 
 1   player_ID  12654 non-null  object 
 2   year_ID    12654 non-null  int64  
 3   team_ID    12654 non-null  object 
 4   lg_ID      12654 non-null  object 
 5   salary     12654 non-null  float64
 6   Age        12340 non-null  float64
 7   G          12340 non-null  float64
 8   AB         12340 non-null  float64
 9   PA         12340 non-null  float64
 10  AVG        12340 non-null  float64
 11  H          12340 non-null  float64
 12  1B         12340 non-null  float64
 13  2B         12340 non-null  float64
 14  3B         12340 non-null  float64
 15  HR         12340 non-null  float64
 16  R          12340 non-null  float64
 17  RBI        12340 non-null  float64
 18  BB         12340 non-null  float64
 19  IBB        12340 non-null  float64
 20  SO    

In [479]:
# check missing values
batting_basic.isna().sum()

name           0
player_ID      0
year_ID        0
team_ID        0
lg_ID          0
salary         0
Age          314
G            314
AB           314
PA           314
AVG          314
H            314
1B           314
2B           314
3B           314
HR           314
R            314
RBI          314
BB           314
IBB          314
SO           314
HBP          314
SF           314
SH           314
GDP          314
SB           314
CS           314
BB%          314
K%           314
BB/K         314
OBP          314
SLG          314
OPS          314
ISO          314
BABIP        314
dtype: int64

**Handling missing values in `batting_basic`**

There are 314 missing values for standard batting statistics which is strange. When checking these missing values in `batting_basic`, there are fairly well-known players with missing statistics. This may be attributed to a scraping bug or data collection error on FanGraphs. 

While we could theoretically look up these players statistics, we'll just drop these missing rows from `batting_basic`. 

In [480]:
# drop all missing nan rows
batting_basic = batting_basic.dropna()

In [481]:
# check
batting_basic.isna().sum()

name         0
player_ID    0
year_ID      0
team_ID      0
lg_ID        0
salary       0
Age          0
G            0
AB           0
PA           0
AVG          0
H            0
1B           0
2B           0
3B           0
HR           0
R            0
RBI          0
BB           0
IBB          0
SO           0
HBP          0
SF           0
SH           0
GDP          0
SB           0
CS           0
BB%          0
K%           0
BB/K         0
OBP          0
SLG          0
OPS          0
ISO          0
BABIP        0
dtype: int64

In [482]:
batting_basic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12340 entries, 0 to 12653
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       12340 non-null  object 
 1   player_ID  12340 non-null  object 
 2   year_ID    12340 non-null  int64  
 3   team_ID    12340 non-null  object 
 4   lg_ID      12340 non-null  object 
 5   salary     12340 non-null  float64
 6   Age        12340 non-null  float64
 7   G          12340 non-null  float64
 8   AB         12340 non-null  float64
 9   PA         12340 non-null  float64
 10  AVG        12340 non-null  float64
 11  H          12340 non-null  float64
 12  1B         12340 non-null  float64
 13  2B         12340 non-null  float64
 14  3B         12340 non-null  float64
 15  HR         12340 non-null  float64
 16  R          12340 non-null  float64
 17  RBI        12340 non-null  float64
 18  BB         12340 non-null  float64
 19  IBB        12340 non-null  float64
 20  SO    

**Save `batting_basic` as a csv**

In [483]:
path = "Data/"

batting_basic.to_csv(path + 'batting_basic')

**Checking and cleaning `batting_adv`:**

In [484]:
batting_adv.head(3)

,name,player_ID,year_ID,team_ID,lg_ID,salary,Age,G,AB,PA,AVG,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%
0,Jeff Abbott,abbotje01,2000,CHW,AL,255000.0,27.0,80.0,215.0,242.0,0.274,59.0,40.0,15.0,1.0,3.0,31.0,29.0,21.0,1.0,38.0,2.0,1.0,2.0,2.0,2.0,1.0,0.087,0.157,0.55,0.343,0.395,0.738,0.121,0.320,-12.6,-6.7,-7.3,0.84,-1.13,-0.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087,0.157,0.55,0.343,0.395,0.738,0.121,0.320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.328,-2.7,29.0,-4.2,-5.0,7.8,-1.6,-2.1,3.9,87.0,0.37,-3.89,4.26,-6.74,-0.57,0.90,1.79,14.0,-0.16,0.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.6,0.0,NaN,27 - 27,-4.2,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0,91.0,100.0,98.0,89.0,72.0,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,Kurt Abbott,abbotku01,2000,NYM,NL,500000.0,31.0,79.0,157.0,173

In [485]:
# check missing values
batting_adv.isna().sum()

name            0
player_ID       0
year_ID         0
team_ID         0
lg_ID           0
             ... 
HardHit      4268
HardHit%     8877
Events        314
CStr%        1477
CSW%         1477
Length: 330, dtype: int64

**Handling missing values in `batting_adv`**

Unfortunately, we have many missing values in the `batting_adv` table. This is expected, since there is a lot of variance when it comes to collecting advanced StatCast data. 

Lets limit `batting_adv` to data between 2014 and 2021 and fill in the remaining missing values with 0s. 

In [486]:
batting_adv = batting_adv[batting_adv['year_ID'] >= 2014]

In [487]:
# remove rows where stats were not scraped properly (same as in batting_basic table)
# rows where 
batting_adv = batting_adv[batting_adv['Age'].notna()]

In [488]:
# fill all missing NaNs with 0
batting_adv = batting_adv.fillna(0)

REMOVE AGE RNG COLUMN

In [513]:
batting_adv[batting_adv['name'] == 'Aaron Judge']

,name,player_ID,year_ID,team_ID,lg_ID,salary,Age,G,AB,PA,AVG,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%
6043,Aaron Judge,judgeaa01,2016,NYY,AL,544500.0,24.0,27.0,84.0,95.0,0.179,15.0,9.0,2.0,0.0,4.0,10.0,10.0,9.0,0.0,42.0,1.0,1.0,0.0,2.0,0.0,1.0,0.095,0.442,0.21,0.263,0.345,0.608,0.167,0.282,-6.1,-6.1,-0.9,0.33,-0.62,-0.29,15.0,22.0,6.0,0.0,408.0,149.0,259.0,1.0,0.0,0.0,0.095,0.442,0.21,0.263,0.345,0.608,0.167,0.282,0.0068,0.140,0.349,0.512,0.000,0.182,0.067,0.0,0.267,-4.0,7.0,-4.3,1.2,2.9,-1.1,-1.5,2.1,62.0,-0.20,-1.38,1.18,-4.71,-0.49,0.75,0.02,1.0,-0.39,0.12,0.498,92.4,0.172,84.2,0.049,87.7,0.098,76.2,0.135,83.0,0.020,81.8,0.029,78.2,0.000,0.0,-1.8,-1.1,0.1,-0.8,0.0,-0.8,0.1,-0.88,-1.61,0.31,-2.00,0.03,-9.95,0.47,0.336,0.634,0.456,0.366,0.788,0.602,0.402,0.579,0.181,-0.5,0.360,0.108,0.049,0.027,0.0,0.037,0.164,0.083,0.017,0.000,0.125,0.0,0.029,0.0,92.6,91.3,88.9,83.6,0.0,92.0,83.8,75.9,78.7,0.0,82.6,0.0,78.1,-2.0,-6.0,1.8,-6.4,0.0,-8.5,1.3,4.5,4.8,0.0,1.9,0.0,-3.3,9.9,7.2,3.9,1.8,0.0,4.5,1.4,-6.5,-6.2,0.0,4.5,0.0,1.8,-2.7,0.1,-0.3,-0.7,0.0,-0.2,-0.1,-0.9,0.1,0.0,0.1,0.0,0.1,-1.85,0.22,-1.49,-6.11,0.0,-1.35,-0.15,-2.57,1.71,0.00,0.18,0.0,0.47,0.311,0.629,0.456,0.377,0.726,0.597,0.456,21.3,0.1,-0.5,-0.1,24 - 24,-4.8,0.4,0.1,0.465,0.326,0.209,0.093,0.419,0.488,0.579,0.120,0.002,0.110,0.360,0.074,0.034,0.029,0.0,0.132,0.137,0.000,83.1,72.9,77.2,93.5,88.9,84.5,78.7,0.0,91.7,84.7,0.0,2.4,10.0,4.8,-1.7,2.1,-6.1,-3.3,0.0,-7.3,1.8,0.0,2.6,-9.0,-7.6,8.7,3.1,1.9,0.2,0.0,5.1,-0.1,0.0,0.0,0.1,-1.5,-1.7,-0.2,-0.8,0.1,0.0,0.1,-0.5,0.0,0.05,5.97,-3.34,-1.16,-0.63,-5.75,0.47,0.0,0.19,-0.96,

**Save `batting_adv` as a csv**

In [490]:
path = "Data/"

batting_adv.to_csv(path + 'batting_adv')

## Formatting the Pitching Table

**Join `pitching_2021` with `bwar_pitch_2021` on the following columns:**

**`pitching_2021`** 
- `Name`
- `Season`

**`bwar_pitch_2021`**  
- `name_common`
- `year_ID`

In [491]:
# join pitching_2021 onto bwar_pitch_2021 (left join)
pitching_merged = pd.merge(bwar_pitch_2021, pitching_2021, how='left',left_on=['name_common','year_ID'],right_on=['Name','Season'])

In [492]:
# check
pitching_merged

,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,G_x,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR_x,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg,IDfg,Season,Name,Team,Age,W,L,WAR_y,ERA,G_y,GS_y,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,Dollars,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,Clutch,FB% 2,FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,HLD,SD,MD,ERA-,FIP-,xFIP-,K%,BB%,SIERA,RS/9,E-F,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,RA9-WAR,BIP-Wins,LOB-Wins,FDP-Wins,Age Rng,K-BB%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,kwERA,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,K/9+,BB/9+,K/BB+,H/9+,HR/9+,AVG+,WHIP+,BABIP+,LOB%+,K%+,BB%+,LD%+,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
0,David Aardsma,22.0,430911.0,aardsda01,2004,SFG,1,NL,11,0,32,0.0,32.0,8,5.967,-0.440,NaN,-0.423,101,98.525,5.862,44.0,0.0093,-45.5,-2.138,-2.177,-1.199,0.203,1.0718,-0.15,300000.0,4.67092,4.86883,1.902,0.4803,-0.2167,-0.0298,4.75633,1.895,0.4914,0.0942,66.675000,5.334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

**Filling missing `salary` values.**

Similar to filling in missing `salary` values for the batting table, we will apply the same function to the `pitching_merged` table. 

To fill in these missing values, we will assume the following:
- A player's salary is the same across consecutive seasons even if the salary is missing
- Players traded mid-season will have the same salary for that season across both teams


In [493]:
# make a copy and specify which column to fill missing values
salary_col = ['salary']
pitching_filled = pitching_merged.copy()

In [494]:
# forward and backward fills missing values grouped by name
pitching_filled.loc[:,salary_col] = pitching_merged.loc[:,salary_col].groupby(pitching_filled['name_common']).ffill().bfill()

In [495]:
# check
pitching_filled['salary'].isna().sum()

0

In [496]:
# check
pitching_merged[['name_common', 'year_ID', 'salary']].head(15)

,name_common,year_ID,salary
0,David Aardsma,2004,300000.0
1,David Aardsma,2006,NaN
2,David Aardsma,2007,387500.0
3,David Aardsma,2008,403250.0
4,David Aardsma,2009,419000.0
5,David Aardsma,2010,2750000.0
6,David Aardsma,2012,500000.0
7,David Aardsma,2013,NaN
8,David Aardsma,2015,NaN
9,Fernando Abad,2010,NaN


In [497]:
# check
pitching_filled[['name_common', 'year_ID', 'salary']].head(15)

,name_common,year_ID,salary
0,David Aardsma,2004,300000.0
1,David Aardsma,2006,300000.0
2,David Aardsma,2007,387500.0
3,David Aardsma,2008,403250.0
4,David Aardsma,2009,419000.0
5,David Aardsma,2010,2750000.0
6,David Aardsma,2012,500000.0
7,David Aardsma,2013,500000.0
8,David Aardsma,2015,500000.0
9,Fernando Abad,2010,418000.0


In [498]:
pitching_filled

,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,G_x,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR_x,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg,IDfg,Season,Name,Team,Age,W,L,WAR_y,ERA,G_y,GS_y,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,Dollars,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,Clutch,FB% 2,FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,HLD,SD,MD,ERA-,FIP-,xFIP-,K%,BB%,SIERA,RS/9,E-F,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,RA9-WAR,BIP-Wins,LOB-Wins,FDP-Wins,Age Rng,K-BB%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,kwERA,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,K/9+,BB/9+,K/BB+,H/9+,HR/9+,AVG+,WHIP+,BABIP+,LOB%+,K%+,BB%+,LD%+,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
0,David Aardsma,22.0,430911.0,aardsda01,2004,SFG,1,NL,11,0,32,0.0,32.0,8,5.967,-0.440,NaN,-0.423,101,98.525,5.862,44.0,0.0093,-45.5,-2.138,-2.177,-1.199,0.203,1.0718,-0.15,300000.0,4.67092,4.86883,1.902,0.4803,-0.2167,-0.0298,4.75633,1.895,0.4914,0.0942,66.675000,5.334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## Simplify and separate the `pitching_filled` table

Similar to what we did above to the `batting_filled` table, we will now simplify into two separate tables for `pitching_filled`. 

Thus, we will create two separate pitching tables:
- `pitching_basic` (containing basic data between 2000 - 2021)
- `pitching_adv` (containing basic AND StatCast data between 2014 - 2021)

We'll also remove repetitive columns that we brought in during the initial merge of tables. 

In [499]:
pitching_filled.head(3)

,name_common,age,mlb_ID,player_ID,year_ID,team_ID,stint_ID,lg_ID,G_x,GS_x,IPouts,IPouts_start,IPouts_relief,RA,xRA,xRA_sprp_adj,xRA_extras_adj,xRA_def_pitcher,PPF,PPF_custom,xRA_final,BIP,BIP_perc,RS_def_total,runs_above_avg,runs_above_avg_adj,runs_above_rep,RpO_replacement,GR_leverage_index_avg,WAR_x,salary,teamRpG,oppRpG,pyth_exponent,waa_win_perc,WAA,WAA_adj,oppRpG_rep,pyth_exponent_rep,waa_win_perc_rep,WAR_rep,ERA_plus,ER_lg,IDfg,Season,Name,Team,Age,W,L,WAR_y,ERA,G_y,GS_y,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,Dollars,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,Clutch,FB% 2,FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,HLD,SD,MD,ERA-,FIP-,xFIP-,K%,BB%,SIERA,RS/9,E-F,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,RA9-WAR,BIP-Wins,LOB-Wins,FDP-Wins,Age Rng,K-BB%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,kwERA,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,K/9+,BB/9+,K/BB+,H/9+,HR/9+,AVG+,WHIP+,BABIP+,LOB%+,K%+,BB%+,LD%+,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
0,David Aardsma,22.0,430911.0,aardsda01,2004,SFG,1,NL,11,0,32,0.0,32.0,8,5.967,-0.440,NaN,-0.423,101,98.525,5.862,44.0,0.0093,-45.5,-2.138,-2.177,-1.199,0.203,1.0718,-0.15,300000.0,4.67092,4.86883,1.902,0.4803,-0.2167,-0.0298,4.75633,1.895,0.4914,0.0942,66.675000,5.334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [500]:
# get all column names
with pd.option_context('display.max_seq_items', None):
    print (pitching_filled.columns)

Index(['name_common', 'age', 'mlb_ID', 'player_ID', 'year_ID', 'team_ID',
       'stint_ID', 'lg_ID', 'G_x', 'GS_x', 'IPouts', 'IPouts_start',
       'IPouts_relief', 'RA', 'xRA', 'xRA_sprp_adj', 'xRA_extras_adj',
       'xRA_def_pitcher', 'PPF', 'PPF_custom', 'xRA_final', 'BIP', 'BIP_perc',
       'RS_def_total', 'runs_above_avg', 'runs_above_avg_adj',
       'runs_above_rep', 'RpO_replacement', 'GR_leverage_index_avg', 'WAR_x',
       'salary', 'teamRpG', 'oppRpG', 'pyth_exponent', 'waa_win_perc', 'WAA',
       'WAA_adj', 'oppRpG_rep', 'pyth_exponent_rep', 'waa_win_perc_rep',
       'WAR_rep', 'ERA_plus', 'ER_lg', 'IDfg', 'Season', 'Name', 'Team', 'Age',
       'W', 'L', 'WAR_y', 'ERA', 'G_y', 'GS_y', 'CG', 'ShO', 'SV', 'BS', 'IP',
       'TBF', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'HBP', 'WP', 'BK', 'SO', 'GB',
       'FB', 'LD', 'IFFB', 'Balls', 'Strikes', 'Pitches', 'RS', 'IFH', 'BU',
       'BUH', 'K/9', 'BB/9', 'K/BB', 'H/9', 'HR/9', 'AVG', 'WHIP', 'BABIP',
       'LOB%', 'FIP', '

**Specify columns for `pitching_basic`**

In [501]:
# basic pitching columns to keep
pitching_basic_columns = ['name_common', 'player_ID', 'year_ID', 'team_ID', 'lg_ID',
                         'salary', 'Age', 'W', 'L', 'WAR_y', 'ERA', 'G_y', 'GS_y', 
                          'CG', 'ShO', 'SV', 'BS', 'IP', 'TBF', 'H', 'RA', 'ER', 'HR', 
                          'BB', 'IBB', 'HBP', 'WP', 'BK', 'SO', 'GB','FB', 'LD', 'IFFB',
                          'Balls', 'Strikes', 'Pitches', 'RS', 'IFH', 'BU','BUH', 'K/9', 
                          'BB/9', 'K/BB', 'H/9', 'HR/9', 'AVG', 'WHIP', 'BABIP','LOB%', 
                          'FIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%',
                          'BUH%']

# create new pitching_basic table with specified columns i want
pitching_basic = pitching_filled[pitching_basic_columns]

# renaming columns
pitching_basic = pitching_basic.rename(columns = {'name_common':'name',
                                                  'WAR_y': 'WAR',
                                                  'G_y':'G',
                                                  'GS_y':'GS'})

**Specify columns for `pitching_adv`**

In [502]:
# adv pitching columns to keep
pitching_adv_columns = ['name_common', 'player_ID', 'year_ID', 'team_ID', 'lg_ID',
                           'salary', 'Age', 'W', 'L', 'WAR_y', 'ERA', 'G_y', 'GS_y', 
                           'CG', 'ShO', 'SV', 'BS', 'IP', 'TBF', 'H', 'RA', 'ER', 'HR', 
                           'BB', 'IBB', 'HBP', 'WP', 'BK', 'SO', 'GB','FB', 'LD', 'IFFB',
                           'Balls', 'Strikes', 'Pitches', 'RS', 'IFH', 'BU','BUH', 'K/9', 
                           'BB/9', 'K/BB', 'H/9', 'HR/9', 'AVG', 'WHIP', 'BABIP','LOB%', 
                           'FIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%',
                           'BUH%', 'xRA', 'xRA_sprp_adj', 'xRA_extras_adj','xRA_def_pitcher', 
                           'PPF', 'PPF_custom', 'xRA_final', 'BIP', 'BIP_perc', 'RS_def_total', 
                           'runs_above_avg', 'runs_above_avg_adj', 'runs_above_rep', 'RpO_replacement', 
                           'GR_leverage_index_avg','ERA_plus','tERA', 'xFIP', 'WPA', '-WPA', '+WPA', 
                           'RE24', 'REW', 'pLI', 'inLI', 'gmLI', 'exLI', 'Pulls', 'WPA/LI', 'Clutch', 'FB% 2', 'FBv',
                           'SL%', 'SLv', 'CT%', 'CTv', 'CB%', 'CBv', 'CH%', 'CHv', 'SF%', 'SFv',
                           'KN%', 'KNv', 'XX%', 'PO%', 'wFB', 'wSL', 'wCT', 'wCB', 'wCH', 'wSF',
                           'wKN', 'wFB/C', 'wSL/C', 'wCT/C', 'wCB/C', 'wCH/C', 'wSF/C', 'wKN/C',
                           'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%', 'Z-Contact%',
                           'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'HLD', 'SD', 'MD', 'ERA-',
                           'FIP-', 'xFIP-', 'K%', 'BB%', 'SIERA', 'RS/9', 'E-F', 'FA% (sc)',
                           'FT% (sc)', 'FC% (sc)', 'FS% (sc)', 'FO% (sc)', 'SI% (sc)', 'SL% (sc)',
                           'CU% (sc)', 'KC% (sc)', 'EP% (sc)', 'CH% (sc)', 'SC% (sc)', 'KN% (sc)',
                           'UN% (sc)', 'vFA (sc)', 'vFT (sc)', 'vFC (sc)', 'vFS (sc)', 'vFO (sc)',
                           'vSI (sc)', 'vSL (sc)', 'vCU (sc)', 'vKC (sc)', 'vEP (sc)', 'vCH (sc)',
                           'vSC (sc)', 'vKN (sc)', 'FA-X (sc)', 'FT-X (sc)', 'FC-X (sc)',
                           'FS-X (sc)', 'FO-X (sc)', 'SI-X (sc)', 'SL-X (sc)', 'CU-X (sc)',
                           'KC-X (sc)', 'EP-X (sc)', 'CH-X (sc)', 'SC-X (sc)', 'KN-X (sc)',
                           'FA-Z (sc)', 'FT-Z (sc)', 'FC-Z (sc)', 'FS-Z (sc)', 'FO-Z (sc)',
                           'SI-Z (sc)', 'SL-Z (sc)', 'CU-Z (sc)', 'KC-Z (sc)', 'EP-Z (sc)',
                           'CH-Z (sc)', 'SC-Z (sc)', 'KN-Z (sc)', 'wFA (sc)', 'wFT (sc)',
                           'wFC (sc)', 'wFS (sc)', 'wFO (sc)', 'wSI (sc)', 'wSL (sc)', 'wCU (sc)',
                           'wKC (sc)', 'wEP (sc)', 'wCH (sc)', 'wSC (sc)', 'wKN (sc)',
                           'wFA/C (sc)', 'wFT/C (sc)', 'wFC/C (sc)', 'wFS/C (sc)', 'wFO/C (sc)',
                           'wSI/C (sc)', 'wSL/C (sc)', 'wCU/C (sc)', 'wKC/C (sc)', 'wEP/C (sc)',
                           'wCH/C (sc)', 'wSC/C (sc)', 'wKN/C (sc)', 'O-Swing% (sc)',
                           'Z-Swing% (sc)', 'Swing% (sc)', 'O-Contact% (sc)', 'Z-Contact% (sc)',
                           'Contact% (sc)', 'Zone% (sc)', 'Pace', 'RA9-WAR', 'BIP-Wins',
                           'LOB-Wins', 'FDP-Wins', 'Age Rng', 'K-BB%', 'Pull%', 'Cent%', 'Oppo%',
                           'Soft%', 'Med%', 'Hard%', 'kwERA', 'TTO%', 'CH% (pi)', 'CS% (pi)',
                           'CU% (pi)', 'FA% (pi)', 'FC% (pi)', 'FS% (pi)', 'KN% (pi)', 'SB% (pi)',
                           'SI% (pi)', 'SL% (pi)', 'XX% (pi)', 'vCH (pi)', 'vCS (pi)', 'vCU (pi)',
                           'vFA (pi)', 'vFC (pi)', 'vFS (pi)', 'vKN (pi)', 'vSB (pi)', 'vSI (pi)',
                           'vSL (pi)', 'vXX (pi)', 'CH-X (pi)', 'CS-X (pi)', 'CU-X (pi)',
                           'FA-X (pi)', 'FC-X (pi)', 'FS-X (pi)', 'KN-X (pi)', 'SB-X (pi)',
                           'SI-X (pi)', 'SL-X (pi)', 'XX-X (pi)', 'CH-Z (pi)', 'CS-Z (pi)',
                           'CU-Z (pi)', 'FA-Z (pi)', 'FC-Z (pi)', 'FS-Z (pi)', 'KN-Z (pi)',
                           'SB-Z (pi)', 'SI-Z (pi)', 'SL-Z (pi)', 'XX-Z (pi)', 'wCH (pi)',
                           'wCS (pi)', 'wCU (pi)', 'wFA (pi)', 'wFC (pi)', 'wFS (pi)', 'wKN (pi)',
                           'wSB (pi)', 'wSI (pi)', 'wSL (pi)', 'wXX (pi)', 'wCH/C (pi)',
                           'wCS/C (pi)', 'wCU/C (pi)', 'wFA/C (pi)', 'wFC/C (pi)', 'wFS/C (pi)',
                           'wKN/C (pi)', 'wSB/C (pi)', 'wSI/C (pi)', 'wSL/C (pi)', 'wXX/C (pi)',
                           'O-Swing% (pi)', 'Z-Swing% (pi)', 'Swing% (pi)', 'O-Contact% (pi)',
                           'Z-Contact% (pi)', 'Contact% (pi)', 'Zone% (pi)', 'Pace (pi)', 'FRM',
                           'K/9+', 'BB/9+', 'K/BB+', 'H/9+', 'HR/9+', 'AVG+', 'WHIP+', 'BABIP+',
                           'LOB%+', 'K%+', 'BB%+', 'LD%+', 'GB%+', 'FB%+', 'HR/FB%+', 'Pull%+',
                           'Cent%+', 'Oppo%+', 'Soft%+', 'Med%+', 'Hard%+', 'EV', 'LA', 'Barrels',
                           'Barrel%', 'maxEV', 'HardHit', 'HardHit%', 'Events', 'CStr%', 'CSW%',
                           'xERA']

# create new batting_adv table with specified columns i want
pitching_adv = pitching_filled[pitching_adv_columns]

# renaming columns
pitching_adv = pitching_adv.rename(columns = {'name_common':'name',
                                              'WAR_y': 'WAR',
                                              'G_y':'G',
                                              'GS_y':'GS'})

**Checking and cleaning `pitching_basic` first:**

In [503]:
# check
pitching_basic.head()

,name,player_ID,year_ID,team_ID,lg_ID,salary,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,RA,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%
0,David Aardsma,aardsda01,2004,SFG,NL,300000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,David Aardsma,aardsda01,2006,CHC,NL,300000.0,24.0,3.0,0.0,-0.2,4.08,45.0,0.0,0.0,0.0,0.0,0.0,53.0,225.0,41.0,25,24.0,9.0,28.0,0.0,1.0,1.0,0.0,49.0,54.0,64.0,28.0,5.0,361.0,601.0,962.0,33.0,3.0,1.0,0.0,8.32,4.75,1.75,6.96,1.53,0.209,1.30,0.232,0.784,5.15,0.84,0.192,0.370,0.438,0.078,0.141,0.056,0.0
2,David Aardsma,aardsda01,2007,CHW,AL,387500.0,25.0,2.0,1.0,0.2,6.40,25.0,0.0,0.0,0.0,0.0,3.0,32.1,151.0,39.0,24,23.0,4.0,17.0,3.0,1.0,2.0,0.0,36.0,35.0,40.0,20.0,2.0,244.0,364.0,608.0,10.0,3.0,2.0,0.0,10.02,4.73,2.12,10.86,1.11,0.293,1.73,0.376,0.642,4.29,0.88,0.211,0.368,0.421,0.050,0.100,0.086,0.0
3,David Aardsma,aardsda01,2008,BOS,AL,403250.0,26.0,4.0,2.0,-0.1,5.55,47.0,0.0,0.0,0.0,0.0,1.0,48.2,228.0,49.0,32,30.0,4.0,35.0,2.0,5.0,3.0,0.0,49.0,59.0,51.0,24.0,7.0,397.0,573.0,970.0,27.0,1.0,5.0,2.0,9.06,6.47,1.40,9.06,0.74,0.261,1.73,0.333,0.683,4.65,1.16,0.179,0.440,0.381,0.137,0.078,0.017,0.4
4,David Aardsma,aardsda01,2009,SEA,AL,419000.0,27.0,3.0,6.0,2.0,2.52,73.0,0.0,0.0,0.0,38.0,4.0,71.1,296.0,49.0,23,20.0,4.0,34.0,3.0,0.0,2.0,0.0,80.0,45.0,96.0,37.0,8.0,494.0,775.0,1269.0,7.0,4.0,3.0,0.0,10.09,4.29,2.35,6.18,0.50,0.187,1.16,0.253,0.775,3.01,0.47,0.208,0.253,0.539,0.083,0.042,0.089,0.0


In [504]:
pitching_basic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16575 entries, 0 to 16574
Data columns (total 58 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       16575 non-null  object 
 1   player_ID  16575 non-null  object 
 2   year_ID    16575 non-null  int64  
 3   team_ID    16575 non-null  object 
 4   lg_ID      16575 non-null  object 
 5   salary     16575 non-null  float64
 6   Age        11510 non-null  float64
 7   W          11510 non-null  float64
 8   L          11510 non-null  float64
 9   WAR        11510 non-null  float64
 10  ERA        11510 non-null  float64
 11  G          11510 non-null  float64
 12  GS         11510 non-null  float64
 13  CG         11510 non-null  float64
 14  ShO        11510 non-null  float64
 15  SV         11510 non-null  float64
 16  BS         10528 non-null  float64
 17  IP         11510 non-null  float64
 18  TBF        11510 non-null  float64
 19  H          11510 non-null  float64
 20  RA    

In [505]:
# check missing values
pitching_basic.isna().sum()

name            0
player_ID       0
year_ID         0
team_ID         0
lg_ID           0
salary          0
Age          5065
W            5065
L            5065
WAR          5065
ERA          5065
G            5065
GS           5065
CG           5065
ShO          5065
SV           5065
BS           6047
IP           5065
TBF          5065
H            5065
RA              0
ER           5065
HR           5065
BB           5065
IBB          5065
HBP          5065
WP           5065
BK           5065
SO           5065
GB           6047
FB           6047
LD           6047
IFFB         6047
Balls        6047
Strikes      6047
Pitches      6047
RS           6047
IFH          6047
BU           6047
BUH          6047
K/9          5065
BB/9         5065
K/BB         5065
H/9          5065
HR/9         5065
AVG          5065
WHIP         5065
BABIP        5065
LOB%         5065
FIP          5065
GB/FB        6047
LD%          6047
GB%          6047
FB%          6047
IFFB%        6047
HR/FB     

**Handling missing values in `pitching_basic`**

There are 5065 missing values for standard pitching statistics and is strange since there are missing values for age and other basic statistics. 

When checking these missing values in `pitching_basic`, there are fairly well-known players with missing statistics. This may be attributed to a scraping bug or data collection error on FanGraphs. 

While we could theoretically look up these players statistics, we'll just drop these missing rows from `pitching_basic`. 

In [508]:
# drop all missing nan rows
pitching_basic = pitching_basic.dropna()

In [509]:
# check
pitching_basic.isna().sum()

name         0
player_ID    0
year_ID      0
team_ID      0
lg_ID        0
salary       0
Age          0
W            0
L            0
WAR          0
ERA          0
G            0
GS           0
CG           0
ShO          0
SV           0
BS           0
IP           0
TBF          0
H            0
RA           0
ER           0
HR           0
BB           0
IBB          0
HBP          0
WP           0
BK           0
SO           0
GB           0
FB           0
LD           0
IFFB         0
Balls        0
Strikes      0
Pitches      0
RS           0
IFH          0
BU           0
BUH          0
K/9          0
BB/9         0
K/BB         0
H/9          0
HR/9         0
AVG          0
WHIP         0
BABIP        0
LOB%         0
FIP          0
GB/FB        0
LD%          0
GB%          0
FB%          0
IFFB%        0
HR/FB        0
IFH%         0
BUH%         0
dtype: int64

In [510]:
pitching_basic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10528 entries, 1 to 16574
Data columns (total 58 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       10528 non-null  object 
 1   player_ID  10528 non-null  object 
 2   year_ID    10528 non-null  int64  
 3   team_ID    10528 non-null  object 
 4   lg_ID      10528 non-null  object 
 5   salary     10528 non-null  float64
 6   Age        10528 non-null  float64
 7   W          10528 non-null  float64
 8   L          10528 non-null  float64
 9   WAR        10528 non-null  float64
 10  ERA        10528 non-null  float64
 11  G          10528 non-null  float64
 12  GS         10528 non-null  float64
 13  CG         10528 non-null  float64
 14  ShO        10528 non-null  float64
 15  SV         10528 non-null  float64
 16  BS         10528 non-null  float64
 17  IP         10528 non-null  float64
 18  TBF        10528 non-null  float64
 19  H          10528 non-null  float64
 20  RA    

In [512]:
pitching_basic

,name,player_ID,year_ID,team_ID,lg_ID,salary,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,RA,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%
1,David Aardsma,aardsda01,2006,CHC,NL,300000.0,24.0,3.0,0.0,-0.2,4.08,45.0,0.0,0.0,0.0,0.0,0.0,53.0,225.0,41.0,25,24.0,9.0,28.0,0.0,1.0,1.0,0.0,49.0,54.0,64.0,28.0,5.0,361.0,601.0,962.0,33.0,3.0,1.0,0.0,8.32,4.75,1.75,6.96,1.53,0.209,1.30,0.232,0.784,5.15,0.84,0.192,0.370,0.438,0.078,0.141,0.056,0.000
2,David Aardsma,aardsda01,2007,CHW,AL,387500.0,25.0,2.0,1.0,0.2,6.40,25.0,0.0,0.0,0.0,0.0,3.0,32.1,151.0,39.0,24,23.0,4.0,17.0,3.0,1.0,2.0,0.0,36.0,35.0,40.0,20.0,2.0,244.0,364.0,608.0,10.0,3.0,2.0,0.0,10.02,4.73,2.12,10.86,1.11,0.293,1.73,0.376,0.642,4.29,0.88,0.211,0.368,0.421,0.050,0.100,0.086,0.000
3,David Aardsma,aardsda01,2008,BOS,AL,403250.0,26.0,4.0,2.0,-0.1,5.55,47.0,0.0,0.0,0.0,0.0,1.0,48.2,228.0,49.0,32,30.0,4.0,35.0,2.0,5.0,3.0,0.0,49.0,59.0,51.0,24.0,7.0,397.0,573.0,970.0,27.0,1.0,5.0,2.0,9.06,6.47,1.40,9.06,0.74,0.261,1.73,0.333,0.683,4.65,1.16,0.179,0.440,0.381,0.137,0.078,0.017,0.400
4,David Aardsma,aardsda01,2009,SEA,AL,419000.0,27.0,3.0,6.0,2.0,2.52,73.0,0.0,0.0,0.0,38.0,4.0,71.1,296.0,49.0,23,20.0,4.0,34.0,3.0,0.0,2.0,0.0,80.0,45.0,96.0,37.0,8.0,494.0,775.0,1269.0,7.0,4.0,3.0,0.0,10.09,4.29,2.35,6.18,0.50,0.187,1.16,0.253,0.775,3.01,0.47,0.208,0.253,0.539,0.083,0.042,0.089,0.000
5,David Aardsma,aardsda01,2010,SEA,AL,2750000.0,28.0,0.0,6.0,0.3,3.44,53.0,0.0,0.0,0.0,31.0,5.0,49.2,202.0,33.0,19,19.0,5.0,25.0,5.0,2.0,2.0,0.0,49.0,42.0,53.0,23.0,4.0,337.0,530.0,867.0,2.0,0.0,8.0,1.0,8.88,4.53,1.96,5.98,0.91,0.189,1.17,0.231,0.774,4.05,0.79,0.195,0.356,0.449,0.075,0.094,0.000,0.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16568,Joel Zumaya,zumayjo01,2007,DET,AL,410000.0,22.0,2.0,3.0,0.2,4.28,28.0,0.0,0.0,0.0,1.0,4.0,33.2,142.0,23.0,16,16.0,3.0,17.0,2.0,1.0,3.0,0.0,27.0,35.0,46.0,15.0,6.0,211.0,342.0,553.0,8.0,1.0,1.0,0.0,7.22,4.54,1.59,6.15,0.80,0.185,1.19,0.213,0.679,4.40,0.76,0.156,0.365,0.479,0.130,0.065,0.029,0.000
16569,Joel Zumaya,zumayjo01,2008,DET,AL,420000.0,23.0,0.0,2.0,-0.4,3.47,21.0,0.0,0.0,0.0,1.0,4.0,23.1,114.0,24.0,13,9.0,3.0,22.0,4.0,0.0,6.0,0.0,22.0,28.0,28.0,14.0,2.0,192.0,258.0,450.0,12.0,3.0,0.0,0.0,8.49,8.49,1.00,9.26,1.16,0.261,1.97,0.313,0.789,5.75,1.00,0.200,0.400,0.400,0.071,0.107,0.107,0.000
16570,Joel Zumaya,zumayjo01,2009,DET,AL,735000.0,24.0,3.0,3.0,-0.2,4.94,29.0,0.0,0.0,0.0,1.0,6.0,31.0,149.0,34.0,18,17.0,5.0,22.0,3.0,1.0,1.0,0.0,30.0,32.0,48.0,13.0,3.0,215.0,368.0,583.0,19.0,2.0,3.0,1.0,8.71,6.39,1.36,9.87,1.45,0.270,1.81,0.319,0.780,5.48,0.67,0.140,0.344,0.516,0.063,0.104,0.063,0.333
16571,Joel Zumaya,zumayjo01,2010,DET,AL,915000.0,25.0,2.0,1.0,1.1,2.35,31.0,0.0,0.0,0.0,1.0,2.0,38.1,156.0,32.0,13,10.0,1.0,11.0,0.0,0.0,2.0,0.0,34.0,40.0,51.0,17.0,7.0,224.0,373.0,597.0,32.0,3.0,3.0,0.0,7.98,2.58,3.09,7.51,0.23,0.221,1.12,0.282,0.697,2.50,0.78,0.157,0.370,0.472,0.137,0.020,0.075,0.000


**Save `pitching_basic` as a csv**

In [511]:
path = "Data/"

pitching_basic.to_csv(path + 'pitching_basic')

**Checking and cleaning `batting_adv`:**

In [404]:
batting_adv.head(3)

,name,player_ID,year_ID,team_ID,lg_ID,salary,Age,G,AB,PA,AVG,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%
0,Jeff Abbott,abbotje01,2000,CHW,AL,255000.0,27.0,80.0,215.0,242.0,0.274,59.0,40.0,15.0,1.0,3.0,31.0,29.0,21.0,1.0,38.0,2.0,1.0,2.0,2.0,2.0,1.0,0.087,0.157,0.55,0.343,0.395,0.738,0.121,0.320,-12.6,-6.7,-7.3,0.84,-1.13,-0.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087,0.157,0.55,0.343,0.395,0.738,0.121,0.320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.328,-2.7,29.0,-4.2,-5.0,7.8,-1.6,-2.1,3.9,87.0,0.37,-3.89,4.26,-6.74,-0.57,0.90,1.79,14.0,-0.16,0.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.6,0.0,NaN,27 - 27,-4.2,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0,91.0,100.0,98.0,89.0,72.0,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,Kurt Abbott,abbotku01,2000,NYM,NL,500000.0,31.0,79.0,157.0,173

In [405]:
# check missing values
batting_adv.isna().sum()

name            0
player_ID       0
year_ID         0
team_ID         0
lg_ID           0
             ... 
HardHit      4268
HardHit%     8877
Events        314
CStr%        1477
CSW%         1477
Length: 330, dtype: int64

**Handling missing values in `batting_adv`**

Unfortunately, we have many missing values in the `batting_adv` table. This is expected, since there is a lot of variance when it comes to collecting advanced StatCast data. 

Lets limit `batting_adv` to data between 2014 and 2021 and fill in the remaining missing values with 0s. 

In [420]:
batting_adv = batting_adv[batting_adv['year_ID'] >= 2014]

In [421]:
# remove rows where stats were not scraped properly (same as in batting_basic table)
# rows where 
batting_adv = batting_adv[batting_adv['Age'].notna()]

In [422]:
# fill all missing NaNs with 0
batting_adv = batting_adv.fillna(0)

In [423]:
batting_adv

,name,player_ID,year_ID,team_ID,lg_ID,salary,Age,G,AB,PA,AVG,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,runs_above_avg,runs_above_avg_off,runs_above_avg_def,WAR_rep,WAA,WAR,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%
22,Bobby Abreu,abreubo01,2014,NYM,NL,9000000.0,40.0,78.0,133.0,155.0,0.248,33.0,23.0,9.0,0.0,1.0,12.0,14.0,20.0,0.0,21.0,0.0,2.0,0.0,3.0,1.0,0.0,0.129,0.135,0.95,0.342,0.338,0.680,0.090,0.283,-8.4,-1.4,-8.5,0.50,-0.99,-0.49,56.0,33.0,25.0,0.0,647.0,260.0,387.0,0.0,0.0,0.0,0.129,0.135,0.95,0.342,0.338,0.680,0.090,0.283,0.0170,0.219,0.491,0.289,0.000,0.030,0.000,0.00,0.309,-0.1,17.0,0.0,-4.7,4.4,-1.4,-2.0,2.4,100.0,0.23,-3.06,3.28,2.82,0.36,1.26,1.60,43.0,0.26,-0.08,0.637,92.5,0.099,83.8,0.053,88.1,0.079,78.0,0.124,83.7,0.009,87.2,0.000,0.0,0.000,0.0,2.0,-0.1,-0.1,-0.3,-1.2,-0.1,0.0,0.49,-0.13,-0.35,-0.52,-1.45,-0.96,0.00,0.254,0.546,0.377,0.653,0.919,0.816,0.422,0.613,0.070,-0.6,0.321,0.151,0.036,0.017,0.000,0.172,0.125,0.045,0.023,0.002,0.108,0.000,0.000,0.0,93.1,92.1,89.4,86.5,0.0,91.6,83.7,77.2,80.1,64.8,83.0,0.0,0.0,-3.2,-7.3,1.5,-5.4,0.0,-7.0,2.4,4.8,5.9,6.1,-7.2,0.0,0.0,8.0,5.2,5.5,2.5,0.0,4.5,-0.2,-7.4,-8.5,-12.6,3.3,0.0,0.0,-1.2,2.2,-0.7,-0.3,0.0,1.1,0.5,-1.1,0.3,-0.1,-0.5,0.0,0.0,-0.56,2.20,-2.95,-3.06,0.00,1.03,0.60,-3.63,1.79,-5.40,-0.76,0.00,0.00,0.235,0.534,0.377,0.588,0.927,0.816,0.474,25.8,-6.1,0.0,-0.5,40 - 40,-0.6,0.3,-0.1,0.377,0.342,0.281,0.105,0.614,0.281,0.271,0.097,0.002,0.073,0.413,0.042,0.029,0.000,0.000,0.219,0.110,0.0,84.1,65.7,79.1,93.8,89.7,86.2,0.0,0.0,92.7,84.7,0.0,-7.4,3.1,5.5,-3.5,1.5,-5.7,0.0,0.0,-7.4,2.2,0.0,2.8,-13.7,-8.2,7.2,4.4,1.4,0.0,0.0,4.1,-0.2,0.0,0.7,-0.1,0.2,0.2,-0.1,-0.1,0.0,0.0,0.1,0.1,0.0,1.09,-5.40,0.42,0.06,-0.27,-0.56,0.00,0.0

**Save `batting_adv` as a csv**

In [424]:
path = "Data/"

batting_adv.to_csv(path + 'batting_adv')

## Collect Team Statistics since 1960

Below we will import team batting, pitching, and fielding statistics since 1960. 

1960 will be used as a benchmark as this was approximately the time when baseball started its 'Expansion Era'. In other words, baseball implemented new rules close to what we see to this day and expanded to 30 teams. 

In [245]:
from pybaseball import team_batting

# get each team's seasonal batting stats from 1960 through 2021
team_batting = team_batting(1960, 2021)

In [246]:
team_batting

,teamIDfg,Season,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,11,1996,SEA,28,1736,5668,6518,1625,1018,343,19,245,993,954,670,57,1052,75,58,46,121,90,39,0.287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.103,0.161,0.64,0.366,0.484,0.850,0.197,0.312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.367,178.6,1018,110.7,52.0,207.3,-18.9,389.7,37.1,NaN,4.4,112,5.23,-118.95,124.18,112.18,10.13,1.04,1.85,174.0,6.88,-1.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.1,-1.2,NaN,20 - 34,109.5,39.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103,107,103,105,109,117,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,5,1996,CLE,30,1705,5681,6486,1665,1089,335,23,218,952,904,671,44,844,43,57,34,164,160,50,0.293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.103,0.130,0.80,0.369,0.475,0.844,0.182,0.309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.367,173.9,1009,102.7,-16.0,206.3,-19.0,321.3,30.2,

In [247]:
from pybaseball import team_pitching

# get each team's individual-season-level pitching stats from 1960 through 2021
team_pitching = team_pitching(1960, 2021)

In [248]:
team_pitching

,teamIDfg,Season,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,11,1996,SEA,28,1736,5668,6518,1625,1018,343,19,245,993,954,670,57,1052,75,58,46,121,90,39,0.287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.103,0.161,0.64,0.366,0.484,0.850,0.197,0.312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.367,178.6,1018,110.7,52.0,207.3,-18.9,389.7,37.1,None,4.4,112,5.23,-118.95,124.18,112.18,10.13,1.04,1.85,174.0,6.88,-1.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.1,-1.2,NaN,20 - 34,109.5,39.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103,107,103,105,109,117,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,5,1996,CLE,30,1705,5681,6486,1665,1089,335,23,218,952,904,671,44,844,43,57,34,164,160,50,0.293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.103,0.130,0.80,0.369,0.475,0.844,0.182,0.309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.367,173.9,1009,102.7,-16.0,206.3,-19.0,321.3,30.2

In [249]:
from pybaseball import team_fielding

# get each team's seasonal fielding stats from 1960 through 2021
team_fielding = team_fielding(1960, 2021)

In [250]:
team_fielding

,teamIDfg,Season,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,11,1996,SEA,28,1736,5668,6518,1625,1018,343,19,245,993,954,670,57,1052,75,58,46,121,90,39,0.287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.103,0.161,0.64,0.366,0.484,0.850,0.197,0.312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.367,178.6,1018,110.7,52.0,207.3,-18.9,389.7,37.1,None,4.4,112,5.23,-118.95,124.18,112.18,10.13,1.04,1.85,174.0,6.88,-1.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.1,-1.2,NaN,20 - 34,109.5,39.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103,107,103,105,109,117,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,5,1996,CLE,30,1705,5681,6486,1665,1089,335,23,218,952,904,671,44,844,43,57,34,164,160,50,0.293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.103,0.130,0.80,0.369,0.475,0.844,0.182,0.309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.367,173.9,1009,102.7,-16.0,206.3,-19.0,321.3,30.2